In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

In [2]:
# Prep data
%run data-prep.ipynb

(8523, 13) (5681, 12) (14204, 13)

Frequency of Categories for varible Item_Fat_Content
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Frequency of Categories for varible Item_Type
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

Frequency of Categories for varible Outlet_Location_Type
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

Frequency of Categories for varible Outlet_Size
Medium    4655
Small     3980
High      1553


data-prep.ipynb:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "outputs": [],
data-prep.ipynb:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "source": [


In [3]:
print(train_features.shape)
print(train_outcome.shape)
print(test_features.shape)

(8523, 36)
(8523,)
(5681, 36)


In [8]:
# Define scaler, classifier, and pipeline to use
scaler = MinMaxScaler()
clf = AdaBoostClassifier()
pipe = make_pipeline(scaler, clf)

param_grid = {'adaboostclassifier__n_estimators': np.arange(55, 60)} # np.arange(5, 10)

def train_model(features, target):
    grid = GridSearchCV(pipe, param_grid)
    grid.fit(features, target)
    grid.best_params_
    return grid

In [9]:
IDs = ['Item_Identifier', 'Outlet_Identifier']
train_predictors = train_features.set_index(IDs)
test_predictors = test_features.set_index(IDs)
test_predictors.head()

,,Item_MRP,Item_Visibility,Item_Weight,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,...,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9,MRP_Factor_0,MRP_Factor_1,MRP_Factor_2,MRP_Factor_3
Item_Identifier,Outlet_Identifier,,,,,,,,,,,,,,,,,,,,,
FDW58,OUT049,107.8622,0.007565,20.750,14,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
FDW14,OUT017,87.3198,0.038428,8.300,6,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
NCN55,OUT010,241.7538,0.099575,14.600,15,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
FDQ58,OUT017,155.0340,0.015388,7.315,6,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
FDY38,OUT027,234.2300,0.118599,13.600,28,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [10]:
model = train_model(train_predictors, train_outcome.round())

C:\Users\knasu\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [11]:
preds = model.predict(test_predictors)

In [12]:
preds

array([ 959.,  959.,  176., ...,  959.,  959.,  959.])

In [13]:
test_predictors['Item_Outlet_Sales'] = preds

In [14]:
submission = test_predictors['Item_Outlet_Sales']

In [15]:
submission.to_csv('submission_ada_boost_gridsearch.csv', header=True)